# GridMet

A simple meteorological gridding program.

## developement via ipython notebook, be sure to replace the files in your python path when making changes.

### Theodore Barnhart | University of Colorado Boulder / INSTAAR
### 16 October 2014

Notebook to distribute meteorologic forcing loosely based on Liston and Elder's MicroMet algorithm.

Liston, G. E. and Elder, K.: A meteorological distribution system for high-resolution terrestrial modeling (MicroMet), J. Hydrometeor, 7(2), 217–234, doi:10.1175/JHM486.1, 2006.

In [1]:
import pandas as pd
import numpy as np
import gdal
from gdalconst import GDT_Float32
from gdalconst import GA_ReadOnly
from osgeo import osr

import ProgressBar as pb

In [2]:
class GridMetStation(object):
    name = ""
    elevation = 0
    
    index = 0
    ATmax = 0
    ATmin = 0
    ATave = 0
    P = 0
    
    def __init__(self, name, elevation, index, ATmax, ATmin, ATave, P):
        self.name = name
        self.elevation = elevation
        self.index = index
        self.ATmax = ATmax
        self.ATmin = ATmin
        self.ATave = ATave
        self.P = P

def makeGridMetStation(name, elevation, index, ATmax, ATmin, ATave, P):
    """Create a GridMetStation Object
    
    makeGridMetStation(name, elevation, index, ATmax, ATmin, ATave, P)
    
    Arguments: (All Required)
        name -- The name of the GridMet station being created.
        elevation -- The elevation of the GridMet station.
        index -- A pandas DataFrame DateTime index object the covers the subsequent data.
        ATmax -- A pandas series of maximum daily air temperatures cooresponding to the index [degC].
        ATmin -- A pandas series of minimum daily air temperatures cooresponding to the index [degC].
        ATave -- A pandas series of average daily air temperatures cooresponding to the index [degC].
        P -- A pandas sereis of daily precipitation values cooresponding to the index [mm].
    """
    station = GridMetStation(name, elevation, index, ATmax, ATmin, ATave, P)
    return station

In [3]:
class GridMetLapseRate(object):
    name = ""
    frequency = ""
    index = 0
    Tavelapse = 0
    Plapse = 0
    Tminlapse = 0
    Tmaxlapse = 0
    
    def __init__(self,name,frequency,index,Tavelapse,Plapse,Tminlapse,Tmaxlapse):
        self.name = name
        self.frequency = frequency
        self.index = index
        self.Tavelapse = Tavelapse
        self.Plapse = Plapse
        self.Tminlapse = Tminlapse
        self.Tmaxlapse = Tmaxlapse

In [4]:
def makeGridMetLapseRate(name,GridMetStation1,GridMetStation2,frequency):
    """Make a GridMet LapseRate object.
    
    makeGridMetLapseRate(name,GridMetStation1, GridMetStation2, frequency)
    
    Arguments: (All Required)
        name -- The name of the lapse rate object to be created.
        GridMetStation1 -- The first GridMet station object.
        GridMetStation2 -- The second GridMet station object, the lapse rate will be calculated between these two stations.
        frequency -- The frequency for the lapse rate to be resampled to, either 'm' or 'd'.
    """
    # compute del_z [m]
    delz = GridMetStation1.elevation - GridMetStation2.elevation
    
    # find the longest comon date range
    index1 = GridMetStation1.index[GridMetStation1.index == GridMetStation2.index]
    index2 = GridMetStation2.index[GridMetStation2.index == GridMetStation1.index]
    
    if len(index1) > len(index2):
        index = index1
    else:
        index = index2
    
    del index1
    del index2
    
    n = len(index)
    
    # preallocate
    temp = pd.DataFrame(np.zeros([n,4]),columns=['Tavelapse','Plapse','Tminlapse','Tmaxlapse'], index=index)
    del index
    del n
    
    for day in temp.index:
        
        del_P = GridMetStation1.P[day] - GridMetStation2.P[day] #[mm]
        del_Tave = GridMetStation1.ATave[day] - GridMetStation2.ATave[day] #[degC]
        del_Tmin = GridMetStation1.ATmin[day] - GridMetStation2.ATmin[day] #[degC]
        del_Tmax = GridMetStation1.ATmax[day] - GridMetStation2.ATmax[day] #[degC]
        
        temp.Plapse[day] = del_P/delz # [mm/m]
        temp.Tavelapse[day] = del_Tave/delz # [degc/m]
        temp.Tminlapse = del_Tmin/delz # [degc/m]
        temp.Tmaxlapse = del_Tmax/delz # [degc/m]
    
    temp = temp.resample(frequency, how=np.nanmean) # resample to the given frequency
    
    # set np.nan lapse rates to 0
    temp.loc[np.isnan(temp.Plapse)==True,'Plapse'] = 0.
    temp.loc[np.isnan(temp.Tavelapse)==True,'Tavelapse'] = 0.
    temp.loc[np.isnan(temp.Tminlapse)==True,'Tminlapse'] = 0.
    temp.loc[np.isnan(temp.Tmaxlapse)==True,'Tmaxlapse'] = 0.
    
    return GridMetLapseRate(name,frequency,temp.index, temp.Tavelapse, temp.Plapse, temp.Tminlapse, temp.Tmaxlapse)

In [5]:
def GridMet(rast,patch,GridMetStation,GridMetLapseRate,out,frmt,prfx):
    """Grid data using GridMet.
    
    gridmet(rast,patch,GridMetStation,GridMetLapseRate,out,frmt,prfx)
    
    Arguments: (All Required)
        rast -- The path to the DEM raster for the area to be gridded [m].
        patch -- The path to the RHESSys Patch raster for ascii grid generation.
        GridMetStation -- The GridMet station object to be distributed over the domain.
        GridMetLapseRate -- The GridMet lapse rate object to be distributed over the domain.
        out -- Path where the resulting tiffs or ascii grids will be output.
        frmt -- 'asciiGrid' or 'tiff' denoting what type of output is desired
        prfx -- ascii grid output file prefix for RHESSys integration.  
    """
    
    ## Load the DEM as a numpy array
    temp = gdal.Open(rast, GA_ReadOnly)    
    patch = gdal.Open(patch, GA_ReadOnly)

    if frmt == 'tiff':
        srs = temp.GetProjection() # get the projection of the input data set
        transform = temp.GetGeoTransform() # get the limits and resolution of the input data set

    dem = np.array(temp.GetRasterBand(1).ReadAsArray(), dtype=np.float64) # load the DEM into memory
    m,n = np.shape(dem) # get the dimensions of the DEM
    o = m*n
    dem2 = np.reshape(dem, [o,1]) # linearalize the DEM

    p = len(GridMetStation.index) # get the length of meteo time series 

    if frmt == 'asciiGrid':
        # pull out the patch data
        grid = np.array(patch.GetRasterBand(1).ReadAsArray(), dtype=np.float64)
        grid2 = np.reshape(grid,[(o),1]) # reshape to linearalize
        grid3 = grid2[np.isnan(grid2) != 1] # remove np.NaN values
        dem3 = dem2[np.isnan(dem2) != 1] 

        q = len(grid3) # get the number of patches w/o nans

        grid_out = np.zeros([p+2,q]) # add 2 to the length so the patch index and elevation can be inserted.
        grid_out[:] = np.nan

        # populate the first line with the patch numbers
        grid_out[0,] = grid3.astype(int) # do this in the header instead

        # populate the fourth line with the patch elevations
        grid_out[1,] = dem3

        # 
        Tmax_grid = grid_out.copy()
        Tmin_grid = grid_out.copy()
        Tave_grid = grid_out.copy()
        P_grid = grid_out.copy()

        #del grid_out, dem3, grid3, grid2 # clean up

    del_z = dem2-GridMetStation.elevation # compute elevation differences

    Tave_lapse = np.zeros([m*n,1]) # preallocate temperature and precip grids
    Tave_lapse[:] = np.nan
    P_lapse = Tave_lapse.copy()
    Tmax_lapse = Tave_lapse.copy()
    Tmin_lapse = Tave_lapse.copy()

    Tave_out = np.zeros([m,n]) # preallocate the output grids
    P_out = Tave_out.copy()
    Tmax_out = Tave_out.copy()
    Tmin_out = Tave_out.copy()

    ct = 2 # initialize the counter
    i = 1
    p = pb.ProgressBar(len(GridMetStation.index))

    for day in GridMetStation.index:
        
        dd = str(day.day)
        mm = str(day.month)
        yy = str(day.year)

        if GridMetLapseRate.frequency == 'm':
            lapsematch = yy+'-'+mm
        elif GridMetLapseRate.frequency == 'd':
            lapsematch = yy+'-'+mm+'-'+dd
            
        Tave_lapse = GridMetStation.ATave[day] - (GridMetLapseRate.Tavelapse[lapsematch][0]*del_z)# distribute T based on the daily lapse rate
        Tmax_lapse = GridMetStation.ATmax[day] - (GridMetLapseRate.Tmaxlapse[lapsematch][0]*del_z)
        Tmin_lapse = GridMetStation.ATmin[day] - (GridMetLapseRate.Tminlapse[lapsematch][0]*del_z)

        P_lapse = GridMetStation.P[day] - (GridMetLapseRate.Plapse[lapsematch][0]*del_z) # distribute P based on the daily lapse rate
        P_lapse[P_lapse < 0.] = 0.
        if frmt == 'tiff':

            P_out = np.reshape(P_lapse,[m,n])
            Tave_out = np.reshape(Tave_lapse,[m,n])
            Tmax_out = np.reshape(Tmax_lapse,[m,n])
            Tmin_out = np.reshape(Tmin_lapse,[m,n])


            outpathT = out+'temperature_'+yy+'_'+mm+'_'+dd+'.tif' # generate the temperature output path
            outpathP = out+'precipitation_'+yy+'_'+mm+'_'+dd+'.tif' # generate the precipitation output path

            output_temp = gdal.GetDriverByName('GTiff').Create(outpathT,n,m,1,gdal.GDT_Float32) # open the output file
            output_temp.SetGeoTransform(transform) # set coordinates
            output_temp.SetProjection(srs) # set projection

            output_prec = gdal.GetDriverByName('GTiff').Create(outpathP,n,m,1,gdal.GDT_Float32) # open the output file
            output_prec.SetGeoTransform(transform) # set coordinates
            output_prec.SetProjection(srs) # set projection

            output_temp.GetRasterBand(1).WriteArray(T_out) # write temperature array to raster
            output_prec.GetRasterBand(1).WriteArray(P_out) # write precipitation array to raster

            #http://gis.stackexchange.com/questions/37238/writing-numpy-array-to-raster-file
        if frmt == 'asciiGrid':

            # output the grids 
            Tmax_grid[ct,:] = Tmax_lapse[np.isnan(Tmax_lapse) != 1]
            #print Tmax_grid[ct,]
            #print
            Tmin_grid[ct,:] = Tmin_lapse[np.isnan(Tmin_lapse) != 1]
            #print Tmin_grid[ct,]
            #print
            Tave_grid[ct,:] = Tave_lapse[np.isnan(Tave_lapse) != 1]
            #print Tave_grid[ct,]
            #print
            P_grid[ct,:] = P_lapse[np.isnan(P_lapse) != 1]
            #print P_grid[ct,]

            #print ct
            ct = ct + 1 # increase the counter by one
        p.animate(i)
        i = i + 1
    if frmt == 'asciiGrid':
        print 'Saving asciiGrid files.'
        outpathTave = out+prfx+'.tavg' # generate the temperature output path
        outpathTmax = out+prfx+'.tmax' # generate the temperature output path
        outpathTmin = out+prfx+'.tmin' # generate the temperature output path
        outpathPatch = out+prfx+'.patch' #generate patch output path

        outpathP = out+prfx+'.rain' # generate the precipitation output path

        # populate the first line with the number of cells
        header = str(q)+'\n'+str(GridMetStation.index[0].year)+' '+str(GridMetStation.index[0].month)+' '+str(GridMetStation.index[0].day)+' 1 \n'

        np.savetxt(outpathTave, Tave_grid, delimiter=' ', newline='\n', header=header,comments='', fmt='%6.3f')
        np.savetxt(outpathTmax, Tmax_grid, delimiter=' ', newline='\n', header=header,comments='',fmt='%6.3f')
        np.savetxt(outpathTmin, Tmin_grid, delimiter=' ', newline='\n', header=header,comments='',fmt='%6.3f')
        np.savetxt(outpathP, P_grid, delimiter=' ', newline='\n', header=header,comments='',fmt='%6.3f')
        np.savetxt(outpathPatch, grid3, delimiter=' ', newline='\n',fmt='i')
        print 'Done!'
    

from GridMet import GridMet as gm

strt = '1980-10-01'
nd = '2012-09-30'

niwot1 = pd.read_pickle('/RHESSys/Comocreek/barnhatb/clim/niwot_snotel.pcl')[strt:nd]
c1_P1 = pd.read_pickle('/RHESSys/Comocreek/barnhatb/clim/lapse_rates/C1_chart_prec.pcl')[strt:nd]


for i in xrange(0,len(c1_P1)):
    
    # skip entries that are only one day
    if (c1_P1.ix[i].Qual_days == 1) or (np.isnan(c1_P1.ix[i].Qual_days) == True):
        continue
    else:
        days = c1_P1.ix[i].Qual_days
        strt2 = int(i-(days-1))
        c1_P1.ix[strt2:i+1].P = c1_P1.ix[i].P/float(days)
        c1_P1.ix[strt2:i+1].Qual_days = 1.

c1_T1 = pd.read_pickle('/RHESSys/Comocreek/barnhatb/clim/lapse_rates/C1_chart_temp.pcl')[strt:nd]
d1_P1 = pd.read_pickle('/RHESSys/Comocreek/barnhatb/clim/lapse_rates/D1_prec.pcl')[strt:nd]

for i in xrange(0,len(d1_P1)):
    
    # skip entries that are only one day
    if (d1_P1.ix[i].Qual_days == 1) or (np.isnan(d1_P1.ix[i].Qual_days) == True):
        continue
    else:
        days = d1_P1.ix[i].Qual_days
        strt2 = int(i-(days-1))
        d1_P1.ix[strt2:i+1].P = d1_P1.ix[i].P/float(days)
        d1_P1.ix[strt2:i+1].Qual_days = 1.

d1_T1 = pd.read_pickle('/RHESSys/Comocreek/barnhatb/clim/lapse_rates/D1_temp.pcl')[strt:nd]

niwot = gm.makeGridMetStation('niwot_snotel',3020.,niwot1.index,niwot1.ATmax,niwot1.ATmin,niwot1.ATave,niwot1.precip)
c1 = gm.makeGridMetStation('c1',3018.,c1_T1.index,c1_T1.Tmax,c1_T1.Tmin,c1_T1.Tave,c1_P1.P)
d1 = gm.makeGridMetStation('d1',3743.,d1_T1.index,d1_T1.Tmax,d1_T1.Tmin,d1_T1.Tave,d1_P1.P)

lapse = gm.makeGridMetLapseRate('como',c1,d1,'m')